# Working with Excel

For this module, we will be loading and exporting to Excel. You will learn about:

* Loading Excel files
* Writing Excel files
* Finding missing data
* Summarizing Text data
* Creating a correlation matrix
* Creating scatter plots

## Let's make an Excel File

In [ ]:
import pandas as pd
from urllib.request import urlopen

In [ ]:
names_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names'
fin = urlopen(names_url)
data = fin.read()

In [ ]:
print(data.decode('utf8'))

In [ ]:
names = '''age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
fnlwgt: continuous.
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
education-num: continuous.
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.
capital-gain: continuous.
capital-loss: continuous.
hours-per-week: continuous.
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
salary:'''
names = [col.split(':')[0] for col in names.strip().split('\n')]
names

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
df = pd.read_csv(url, names=names, index_col=False)

In [ ]:
df

In [ ]:
import os
os.makedirs('data', exist_ok=True)

In [ ]:
df.to_excel('data/adult.xlsx')

## Read Excel

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

In [ ]:
!open data/adult.xlsx

In [ ]:
df = pd.read_excel('data/adult.xlsx', dtype_backend='pyarrow')

In [ ]:
df

In [ ]:
df = pd.read_excel('data/adult.xlsx', index_col=0, dtype_backend='pyarrow')

In [ ]:
df

In [ ]:
df.dtypes

## Find Missing Data

In [ ]:
df.isna()

In [ ]:
df.isna().any()

In [ ]:
df.isna().sum()

In [ ]:
df.isna().mean().mul(100)

In [ ]:
(df
 .age
 .gt(50)
 .mean() * 100
 #.mul(100)
)

## Object Columns

In [ ]:
df.select_dtypes('string')  # object in Pandas 1.x

In [ ]:
df.education.value_counts().plot.barh()

In [ ]:
# Filter columns
(df
 .filter(regex=r'education'))

In [ ]:
# value_counts also works with numbers
df.age.value_counts().sort_index().plot.bar()

In [ ]:
df.age.hist(bins=20, figsize=(8,3))

## Numerical Columns

In [ ]:
(df
 .corr(numeric_only=True)
 .style
 .background_gradient(cmap='RdBu', vmin=-1, vmax=1)
)

In [ ]:
(df
 .plot.scatter(x='education-num',
               y='capital-gain',
              figsize=(8,3)))

In [ ]:
(df
 .plot.scatter(x='education-num',
               y='capital-gain', alpha=.3,
               figsize=(8,3)))

In [ ]:
import helpers

(df
 .assign(edu=lambda df_: helpers.jitter(df_, col='education-num'))
 .plot.scatter(x='edu',
               y='capital-gain', alpha=.01,
               figsize=(8,3)))               

In [ ]:
df['education-num'].hist()

## Writing Excel

In [ ]:
!pip install xlsxwriter

In [ ]:
import pandas as pd

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('data/pandas_output.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)
df.query('age < 30').to_excel(writer, sheet_name='LT30', index=False)

# Get the workbook and the worksheet for further manipulation
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Add a header format
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#D7E4BC',  # colors can be set using Hex codes
    'border': 1})

# Write the column headers with the defined format
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num, value, header_format)

# Create a format for the data cells
data_format = workbook.add_format({'text_wrap': True})

# Adjust the width of the columns based on the content
worksheet.autofit()

# Save the excel file
writer.close()


In [ ]:
!open data/pandas_output.xlsx
